# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
X = df.drop(columns = ['renda', 'data_ref','id_cliente'])

In [5]:
X = pd.get_dummies(X, drop_first=True)
X.fillna(0, inplace=True)
print(X.info())
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     15000 non-null  int64  
 1   posse_de_veiculo               15000 non-null  bool   
 2   posse_de_imovel                15000 non-null  bool   
 3   qtd_filhos                     15000 non-null  int64  
 4   idade                          15000 non-null  int64  
 5   tempo_emprego                  15000 non-null  float64
 6   qt_pessoas_residencia          15000 non-null  float64
 7   sexo_M                         15000 non-null  bool   
 8   tipo_renda_Bolsista            15000 non-null  bool   
 9   tipo_renda_Empresário          15000 non-null  bool   
 10  tipo_renda_Pensionista         15000 non-null  bool   
 11  tipo_renda_Servidor público    15000 non-null  bool   
 12  educacao_Pós graduação         15000 non-null 

,Unnamed: 0,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,0,False,True,0,26,6.602740,1.0,False,False,True,...,False,False,True,False,False,True,False,False,False,False
1,1,True,True,0,28,7.183562,2.0,True,False,False,...,False,False,False,False,False,True,False,False,False,False
2,2,True,True,0,35,0.838356,2.0,False,False,True,...,False,False,False,False,False,True,False,False,False,False
3,3,False,True,1,30,4.846575,3.0,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,4,True,False,0,33,4.293151,1.0,True,False,False,...,False,False,True,False,False,False,False,False,False,True


In [6]:
y =  df.loc[:,'renda']

y

0         8060.34
1         1852.15
2         2253.89
3         6600.77
4         6475.97
           ...   
14995     7990.58
14996    10093.45
14997      604.82
14998     3352.27
14999     1160.99
Name: renda, Length: 15000, dtype: float64

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### 1 - Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [7]:
df_train, df_test = train_test_split(df, test_size=0.25,
    train_size=0.75, random_state = 123)

### 2 - Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o  𝑅2 na base de testes. Qual o melhor modelo?

In [8]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = 0)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:47:52   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = 0.001)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:47:57   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = 0.005)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:48:01   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = 0.01)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:48:03   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = 0.05)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:48:04   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
modelo = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = 0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     41.77
Date:                Mon, 29 Apr 2024   Prob (F-statistic):          1.84e-127
Time:                        10:48:04   Log-Likelihood:                -4513.9
No. Observations:                3750   AIC:                             9064.
Df Residuals:                    3733   BIC:                             9176.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            7.7633      0.158     49.147      0.000       7.454       8.073
sexo[T.M]                            0.7161      0.031     23.022      0.000       0.655       0.777
posse_de_veiculo[T.True]            -0.0504      0.029     -1.729      0.084      -0.108       0.007
posse_de_imovel[T.True]              0.0937      0.029      3.265      0.001       0.037       0.150
tipo_renda[T.Empresário]             0.0561      0.034      1.673      0.094      -0.010       0.122
tipo_renda[T.Pensionista]           -0.1432      0.039     -3.716      0.000      -0.219      -0.068
tipo_renda[T.Servidor público]       0.1865      0.048      3.894      0.000       0.093       0.280
educacao[T.Pós graduação]                 0          0        nan        nan           0           0
educacao[T.Secundário]               0.1744      0.129      1.350      0.177      -0.079       0.428
educacao[T.Superior completo]        0.2743      0.130      2.107      0.035       0.019       0.530
educacao[T.Superior incompleto]      0.1398      0.145      0.966      0.334      -0.144       0.424
estado_civil[T.Separado]            -0.0335      0.058     -0.581      0.561      -0.146       0.079
estado_civil[T.Solteiro]            -0.1061      0.042     -2.526      0.012      -0.188      -0.024
estado_civil[T.União]               -0.1008      0.052     -1.953      0.051      -0.202       0.000
estado_civil[T.Viúvo]                0.0075      0.065      0.116      0.908      -0.120       0.135
tipo_residencia[T.Casa]             -0.0252      0.090     -0.280      0.779      -0.201       0.151
tipo_residencia[T.Com os pais]      -0.2032      0.109     -1.870      0.061      -0.416       0.010
tipo_residencia[T.Comunitário]            0          0        nan        nan           0           0
tipo_residencia[T.Estúdio]                0          0        nan        nan           0           0
tipo_residencia[T.Governamental]    -0.0539      0.117     -0.462      0.644      -0.282       0.175
==============================================================================
Omnibus:                       31.825   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.324
Skew:                           0.161   Prob(JB):                     7.86e-09
Kurtosis:                       3.368   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

 - O aplha de 0.1 se demostrou o pior R-quadrado, os outros não tiveram diferença entre si.

### 3 - Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [14]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.001)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:48:07   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.005)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:48:08   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.01)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     20.84
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           2.50e-68
Time:                        10:48:09   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3731   BIC:                         7.793e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2668.0972   1533.309      1.740      0.082    -338.108    5674.303
sexo[T.M]                         5077.9332    296.803     17.109      0.000    4496.021    5659.846
posse_de_veiculo[T.True]          -655.9976    278.909     -2.352      0.019   -1202.826    -109.169
posse_de_imovel[T.True]            331.3011    274.259      1.208      0.227    -206.412     869.014
tipo_renda[T.Empresário]           -39.6330    320.405     -0.124      0.902    -667.820     588.554
tipo_renda[T.Pensionista]        -1378.7169    368.511     -3.741      0.000   -2101.220    -656.213
tipo_renda[T.Servidor público]     998.1774    457.700      2.181      0.029     100.811    1895.544
educacao[T.Pós graduação]         -681.6124   2936.054     -0.232      0.816   -6438.041    5074.816
educacao[T.Secundário]            1236.0527   1402.149      0.882      0.378   -1513.000    3985.105
educacao[T.Superior completo]     1877.1237   1411.676      1.330      0.184    -890.608    4644.855
educacao[T.Superior incompleto]    783.2434   1535.730      0.510      0.610   -2227.708    3794.195
estado_civil[T.Separado]          -682.5155    550.225     -1.240      0.215   -1761.286     396.255
estado_civil[T.Solteiro]          -799.5578    401.106     -1.993      0.046   -1585.966     -13.150
estado_civil[T.União]            -1152.6436    492.993     -2.338      0.019   -2119.205    -186.082
estado_civil[T.Viúvo]             -180.4060    622.557     -0.290      0.772   -1400.991    1040.179
tipo_residencia[T.Casa]            378.4773    617.750      0.613      0.540    -832.683    1589.637
tipo_residencia[T.Com os pais]   -1434.5002    853.949     -1.680      0.093   -3108.752     239.751
tipo_residencia[T.Comunitário]   -1202.1921   2318.494     -0.519      0.604   -5747.831    3343.446
tipo_residencia[T.Estúdio]        1454.0367   1795.126      0.810      0.418   -2065.487    4973.560
tipo_residencia[T.Governamental]          0          0        nan        nan           0           0
==============================================================================
Omnibus:                     5338.400   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080292.424
Skew:                           8.276   Prob(JB):                         0.00
Kurtosis:                     117.192   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.05)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     19.81
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           9.44e-68
Time:                        10:48:10   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3730   BIC:                         7.793e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2063.9715   1790.038      1.153      0.249   -1445.578    5573.521
sexo[T.M]                         5091.9091    297.594     17.110      0.000    4508.447    5675.372
posse_de_veiculo[T.True]          -660.0482    278.999     -2.366      0.018   -1207.053    -113.043
posse_de_imovel[T.True]            338.5497    274.504      1.233      0.218    -199.643     876.742
tipo_renda[T.Empresário]           -34.3659    320.531     -0.107      0.915    -662.799     594.068
tipo_renda[T.Pensionista]        -1388.5388    368.845     -3.765      0.000   -2111.697    -665.380
tipo_renda[T.Servidor público]    1004.8182    457.848      2.195      0.028     107.162    1902.475
educacao[T.Pós graduação]         -680.1186   2936.280     -0.232      0.817   -6436.991    5076.753
educacao[T.Secundário]            1238.6273   1402.262      0.883      0.377   -1510.647    3987.902
educacao[T.Superior completo]     1882.9504   1411.812      1.334      0.182    -885.049    4650.949
educacao[T.Superior incompleto]    802.5686   1536.131      0.522      0.601   -2209.171    3814.308
estado_civil[T.Separado]          -680.9958    550.272     -1.238      0.216   -1759.858     397.867
estado_civil[T.Solteiro]          -779.1296    402.350     -1.936      0.053   -1567.977       9.718
estado_civil[T.União]            -1137.5361    493.571     -2.305      0.021   -2105.231    -169.841
estado_civil[T.Viúvo]             -166.2244    622.982     -0.267      0.790   -1387.643    1055.194
tipo_residencia[T.Casa]            966.4394   1090.637      0.886      0.376   -1171.863    3104.742
tipo_residencia[T.Com os pais]    -849.7784   1236.235     -0.687      0.492   -3273.540    1573.983
tipo_residencia[T.Comunitário]    -614.8720   2486.421     -0.247      0.805   -5489.749    4260.005
tipo_residencia[T.Estúdio]        2037.6702   2004.728      1.016      0.309   -1892.800    5968.141
tipo_residencia[T.Governamental]   851.9159   1302.277      0.654      0.513   -1701.329    3405.161
==============================================================================
Omnibus:                     5338.036   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080165.394
Skew:                           8.275   Prob(JB):                         0.00
Kurtosis:                     117.189   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia'
md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     20.84
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           2.50e-68
Time:                        10:48:11   Log-Likelihood:                -38881.
No. Observations:                3750   AIC:                         7.780e+04
Df Residuals:                    3731   BIC:                         7.793e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         2668.0972   1533.309      1.740      0.082    -338.108    5674.303
sexo[T.M]                         5077.9332    296.803     17.109      0.000    4496.021    5659.846
posse_de_veiculo[T.True]          -655.9976    278.909     -2.352      0.019   -1202.826    -109.169
posse_de_imovel[T.True]            331.3011    274.259      1.208      0.227    -206.412     869.014
tipo_renda[T.Empresário]           -39.6330    320.405     -0.124      0.902    -667.820     588.554
tipo_renda[T.Pensionista]        -1378.7169    368.511     -3.741      0.000   -2101.220    -656.213
tipo_renda[T.Servidor público]     998.1774    457.700      2.181      0.029     100.811    1895.544
educacao[T.Pós graduação]         -681.6124   2936.054     -0.232      0.816   -6438.041    5074.816
educacao[T.Secundário]            1236.0527   1402.149      0.882      0.378   -1513.000    3985.105
educacao[T.Superior completo]     1877.1237   1411.676      1.330      0.184    -890.608    4644.855
educacao[T.Superior incompleto]    783.2434   1535.730      0.510      0.610   -2227.708    3794.195
estado_civil[T.Separado]          -682.5155    550.225     -1.240      0.215   -1761.286     396.255
estado_civil[T.Solteiro]          -799.5578    401.106     -1.993      0.046   -1585.966     -13.150
estado_civil[T.União]            -1152.6436    492.993     -2.338      0.019   -2119.205    -186.082
estado_civil[T.Viúvo]             -180.4060    622.557     -0.290      0.772   -1400.991    1040.179
tipo_residencia[T.Casa]            378.4773    617.750      0.613      0.540    -832.683    1589.637
tipo_residencia[T.Com os pais]   -1434.5002    853.949     -1.680      0.093   -3108.752     239.751
tipo_residencia[T.Comunitário]   -1202.1921   2318.494     -0.519      0.604   -5747.831    3343.446
tipo_residencia[T.Estúdio]        1454.0367   1795.126      0.810      0.418   -2065.487    4973.560
tipo_residencia[T.Governamental]          0          0        nan        nan           0           0
==============================================================================
Omnibus:                     5338.400   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2080292.424
Skew:                           8.276   Prob(JB):                         0.00
Kurtosis:                     117.192   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- Não houve diferenças entre os R-quadrados para o modelo Lasso, porém a partir de 0.005 a variável tipo_renda[T.Bolsista] passou a ser zerada pelo modelo.
- O melhor modelo é o que utiliza a regularização tipo ridge

### 4 - Rode um modelo stepwise. Avalie o  𝑅2 na base de testes. Qual o melhor resultado?

In [19]:
X.replace({False: 0, True: 1}, inplace=True)
X = X.apply(pd.to_numeric, errors='coerce')
X

,Unnamed: 0,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,0,0,1,0,26,6.602740,1.0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,1,1,1,0,28,7.183562,2.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,1,1,0,35,0.838356,2.0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,3,0,1,1,30,4.846575,3.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,1,0,0,33,4.293151,1.0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,0,1,0,48,13.887671,1.0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
14996,14996,0,1,0,57,0.000000,1.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
14997,14997,1,1,0,45,7.832877,2.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
14998,14998,1,0,0,36,4.298630,2.0,1,0,1,...,0,0,0,0,0,1,0,0,0,0


In [20]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 0.0
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Pensionista         with p-value 1.0278e-73
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Pensionista']
Add  tipo_renda_Empresário          with p-value 2.93249e-08
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Pensionista', 'tipo_renda_Empresário']
Add  idade                          with p-value 1.67063e-08
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Pensionista', 'tipo_renda_Empresário', 'idade']
Add  educacao_Superior completo     with p-value 6.30967e-08
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Pensionista', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo']
Add  posse_de_imovel                with p-value 0.00265909
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Pensionista', 'tipo_renda_Empresário', 'idade', 'educacao_Superior c

In [21]:
reg_stepwise = sm.OLS(y, sm.add_constant(pd.DataFrame(X[variaveis]))).fit()
reg_stepwise.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     590.6
Date:                Mon, 29 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:50:45   Log-Likelihood:            -1.5431e+05
No. Observations:               15000   AIC:                         3.086e+05
Df Residuals:                   14990   BIC:                         3.087e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                      -3203.9827    379.398     -8.445      0.000   -3947.649   -2460.316
tempo_emprego                552.8096     10.078     54.855      0.000     533.056     572.563
sexo_M                      5880.6464    128.738     45.679      0.000    5628.303    6132.989
tipo_renda_Pensionista      2659.3978    234.145     11.358      0.000    2200.444    3118.351
tipo_renda_Empresário        793.8169    143.217      5.543      0.000     513.094    1074.540
idade                         44.5258      7.243      6.148      0.000      30.329      58.723
educacao_Superior completo   649.2488    122.594      5.296      0.000     408.950     889.547
posse_de_imovel              361.2204    125.462      2.879      0.004     115.300     607.140
qt_pessoas_residencia        186.9732     68.406      2.733      0.006      52.890     321.057
estado_civil_União          -475.3666    225.733     -2.106      0.035    -917.830     -32.903
==============================================================================
Omnibus:                    21879.133   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         15434292.884
Skew:                           8.577   Prob(JB):                         0.00
Kurtosis:                     159.207   Cond. No.                         309.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

O melhor modelo é o stepwise com um R-quadrado de 0.26